In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import numpy as np
import pandas as pd

In [ ]:
# Data loading and preprocessing
class GlaucomaDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
data_path = "/home/abhaysinghyadav/JustRAIGS_ML/dataset/combined_images/"
df = pd.read_csv("/home/abhaysinghyadav/JustRAIGS_ML/labels.csv", sep=';', low_memory=False)
final_label = df['Final Label'].values

In [ ]:
images = []
labels = final_label
glaucoma_pictures = os.listdir(data_path)

for image in glaucoma_pictures:
    image_path = os.path.join(data_path, image)
    images.append(image_path)

print(len(images))
print(len(glaucoma_pictures))
print(len(labels))

for label in final_label:
    print(label)

# Split data into train and validation sets    
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, stratify=labels)


In [ ]:
# Define data transforms
train_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Create PyTorch datasets
train_dataset = GlaucomaDataset(train_images, train_labels, transform=train_transform)
val_dataset = GlaucomaDataset(val_images, val_labels, transform=val_transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
# Model architecture
class GlaucomaModel(nn.Module):
    def __init__(self):
        super(GlaucomaModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 128 * 128, 16)
        self.fc2 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = GlaucomaModel()

In [ ]:
# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    
    for images, labels in train_loader:
        images = images.to(device)
        if isinstance(labels, tuple):
            labels = tuple(torch.tensor(int(label)).to(device) for label in labels)
        else: 
            labels = torch.tensor(int(labels)).to(device)

        
        labels = labels.float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * images.size(0)
        train_acc += ((outputs > 0.5).float() == labels).sum().item()
    
    train_loss /= len(train_dataset)
    train_acc /= len(train_dataset)
    
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device).float().unsqueeze(1)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            val_loss += loss.item() * images.size(0)
            val_acc += ((outputs > 0.5).float() == labels).sum().item()
    
    val_loss /= len(val_dataset)
    val_acc /= len(val_dataset)
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    

In [ ]:
# Evaluation and metrics
model.eval()
val_preds = []
val_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        preds = (outputs > 0.5).float()
        
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

val_preds = np.array(val_preds)
val_labels = np.array(val_labels)

accuracy = (val_preds == val_labels).mean()
print(f"Validation Accuracy: {accuracy:.4f}")